# Blatt 4

## Aufgabe 10: Zwei Populationen

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd

In [ ]:
mux0 = 0 
muy0 = 3
sigx0 = 3.5
sigy0 = 2.6
cor0 = 0.9
cov0 = cor0 * sigx0 * sigy0

cov_mat0 = np.array([[sigx0**2, cov0], [cov0, sigy0**2]])

In [ ]:
population0_10000 = np.random.multivariate_normal([mux0, muy0], cov_mat0, 10000)

In [ ]:
mux1 = 6
sigx1 = 3.5
a = -0.5 
b = 0.6
var_yx = 1

muy1 = a + b * mux1
sigy1 = np.sqrt(b**2 * sigx1**2 + var_yx)
cor1 = np.sqrt(b**2 * sigx1**2 / sigy1**2)

cov1 = cor1 * sigx1 * sigy1
cov_mat1 = np.array([[sigx1**2, cov1], [cov1, sigy1**2]])

print(muy1, sigy1, cor1)

In [ ]:
population1 = np.random.multivariate_normal([mux1, muy1], cov_mat1, 10000)

__b)__ Zeichne Scatter-Plots:

In [ ]:
plt.scatter(population0_10000[:, 0], population0_10000[:, 1], s=5, alpha=0.2, label = 'Population 0')
plt.scatter(population1[:, 0], population1[:, 1], s=5, alpha=0.2, label = 'Population 1')
plt.legend()

In [ ]:
population0_1000 = np.random.multivariate_normal([mux0, muy0], cov_mat0, 1000)

population0_10000_df = pd.DataFrame({
    'x': population0_10000[:, 0],
    'y': population0_10000[:, 1]
})

population0_1000_df = pd.DataFrame({
    'x': population0_1000[:, 0],
    'y': population0_1000[:, 1]
})

population1_df = pd.DataFrame({
    'x': population1[:, 0],
    'y': population1[:, 1]
})

population0_10000_df.to_hdf('sample.hdf5', key = 'population0_10000')
population0_1000_df.to_hdf('sample.hdf5', key = 'population0_1000')
population1_df.to_hdf('sample.hdf5', key = 'population1')

## Aufgabe 11: Fisher-Diskriminante: Per Hand

siehe Abgabe

## Aufgabe 12: Fisher-Diskriminante: Implementierung

Lade Daten:

In [ ]:
import pandas as pd
P_0 = pd.read_hdf('sample.hdf5', key='population0_10000')
P_1 = pd.read_hdf('sample.hdf5', key='population1')
P_0.head()

__a)__ Berechne Mittelwerte:

In [ ]:
mu0 = np.matrix([P_0.x.mean(), P_0.y.mean()]).T
mu1 = np.matrix([P_1.x.mean(), P_1.y.mean()]).T

__b)__ Berechne Kovarianzmatrizen:

In [ ]:
V_0 = P_0.cov()
V_1 = P_0.cov()

In [ ]:
plt.scatter(P_0.x, P_0.y, s = 5, alpha = 0.2, label = 'Population 0')
plt.scatter(P_1.x, P_1.y, s = 5, alpha = 0.2, label = 'Population 1')
plt.legend()

__c)__ Konstruiere $\vec{\lambda}$:

In [ ]:
S_0 = np.sum([(xi.T - mu0) * (xi.T - mu0).T for xi in np.matrix(P_0)], axis = 0)
S_1 = np.sum([(xi.T - mu1) * (xi.T - mu1).T for xi in np.matrix(P_1)], axis = 0)
S_W = np.matrix(S_0 + S_1)

In [ ]:
lam = S_W.I * (mu1 - mu0)
normed_lam = lam /  np.sqrt(lam[0]**2 + lam[1]**2)
lam_array = np.array([lam[0], lam[1]])[:, 0]
print(lam_array[1] / lam_array[0])

Die Geradengleichung lautet: $$y(x) \approx -1.269 \cdot x$$  

In [ ]:
xplot = np.linspace(-5, 5, 100)
plt.plot(xplot, lam_array[1] / lam_array[0] * xplot,
         color = 'red', label = r'$\vec{\lambda}$')

plt.scatter(P_0.x, P_0.y, s = 5, alpha = 0.2, label = 'Population 0')
plt.scatter(P_1.x, P_1.y, s = 5, alpha = 0.2, label = 'Population 1')
plt.legend()

__d)__ Stelle die Projektionen in einem Histogramm dar:

In [ ]:
projection_0 = np.array([(xi * normed_lam)[0, 0] for xi in np.matrix(P_0)])
projection_1 = np.array([(xi * normed_lam)[0, 0] for xi in np.matrix(P_1)])

In [ ]:
plt.hist(projection_0, histtype = 'step', label = 'Population 0', bins = 25)
plt.hist(projection_1, histtype = 'step', label = 'Population 1', bins = 25)
plt.legend()

__e)__

In [ ]:
def precision(signal, noise, cut):
    true_pos  = np.array([len(signal[signal < cut]) for cut in cut]) 
    false_pos = np.array([len(noise[noise < cut]) for cut in cut])
    return true_pos / (true_pos + false_pos)

In [ ]:
def recall(signal, noise, cut):
    true_pos  = np.array([len(signal[signal < cut]) for cut in cut])
    false_neg = np.array([len(signal[signal > cut]) for cut in cut])
    return true_pos / (true_pos + false_neg)

In [ ]:
signal = projection_0
noise = projection_1

lam_cut = np.linspace(-6, 5, 100)
plt.plot(lam_cut, precision(signal, noise, lam_cut), 
         label = 'Reinheit')
plt.plot(lam_cut, recall(signal, noise, lam_cut), 
         label = 'Effizienz')
plt.legend()

__f)__ Untersuche Signal-Untergrundverhältnis:

In [ ]:
def signal_noise_ratio(signal, noise, cut):
    return np.array([len(signal[signal <= cut]) / len(noise[noise <= cut]) for cut in cut])

In [ ]:
lam_cut = np.linspace(min(noise), 2, 1000)
plt.plot(lam_cut, signal_noise_ratio(signal, noise, lam_cut), 
         label = 'Signal-Untergrundverhältnis')

lam_cut[np.argmax(signal_noise_ratio(signal, noise, lam_cut))]

__g)__ Untersuche Signifikanz:

In [ ]:
def sig(signal, noise, cut):
    return np.array([len(signal[signal <= cut]) / 
                     np.sqrt(len(noise[noise <= cut]) + len(signal[signal <= cut])) for cut in cut])

In [ ]:
lam_cut = np.linspace(-5, 5, 1000)
plt.plot(lam_cut, sig(signal, noise, lam_cut), 
         label = 'Signifikanz')

lam_cut[np.argmax(sig(signal, noise, lam_cut))]

### Nun alles nochmal mit der kleineren Population

Lade Daten:

In [ ]:
import pandas as pd
P_0 = pd.read_hdf('sample.hdf5', key='population0_1000')
P_1 = pd.read_hdf('sample.hdf5', key='population1')
P_0.head()

__a)__ Berechne Mittelwerte:

In [ ]:
mu0 = np.matrix([P_0.x.mean(), P_0.y.mean()]).T
mu1 = np.matrix([P_1.x.mean(), P_1.y.mean()]).T

__b)__ Berechne Kovarianzmatrizen:

In [ ]:
V_0 = P_0.cov()
V_1 = P_0.cov()

In [ ]:
plt.scatter(P_0.x, P_0.y, s = 5, alpha = 0.2, label = 'Population 0')
plt.scatter(P_1.x, P_1.y, s = 5, alpha = 0.2, label = 'Population 1')
plt.legend()

__c)__ Konstruiere $\vec{\lambda}$:

In [ ]:
S_0 = np.sum([(xi.T - mu0) * (xi.T - mu0).T for xi in np.matrix(P_0)], axis = 0)
S_1 = np.sum([(xi.T - mu1) * (xi.T - mu1).T for xi in np.matrix(P_1)], axis = 0)
S_W = np.matrix(S_0 + S_1)

In [ ]:
lam = S_W.I * (mu1 - mu0)
normed_lam = lam /  np.sqrt(lam[0]**2 + lam[1]**2)
lam_array = np.array([lam[0], lam[1]])[:, 0]
print(lam_array[1] / lam_array[0])

Die Geradengleichung lautet: $$y(x) \approx -1.329 \cdot x$$  

In [ ]:
xplot = np.linspace(-5, 5, 100)
plt.plot(xplot, lam_array[1] / lam_array[0] * xplot,
         color = 'red', label = r'$\vec{\lambda}$')

plt.scatter(P_0.x, P_0.y, s = 5, alpha = 0.2, label = 'Population 0')
plt.scatter(P_1.x, P_1.y, s = 5, alpha = 0.2, label = 'Population 1')
plt.legend()

__d)__ Stelle die Projektionen in einem Histogramm dar:

In [ ]:
projection_0 = np.array([(xi * normed_lam)[0, 0] for xi in np.matrix(P_0)])
projection_1 = np.array([(xi * normed_lam)[0, 0] for xi in np.matrix(P_1)])

In [ ]:
plt.hist(projection_0, histtype = 'step', label = 'Population 0', bins = 25)
plt.hist(projection_1, histtype = 'step', label = 'Population 1', bins = 25)
plt.legend()

__e)__

In [ ]:
def precision(signal, noise, cut):
    true_pos  = np.array([len(signal[signal < cut]) for cut in cut]) 
    false_pos = np.array([len(noise[noise < cut]) for cut in cut])
    return true_pos / (true_pos + false_pos)

In [ ]:
def recall(signal, noise, cut):
    true_pos  = np.array([len(signal[signal < cut]) for cut in cut])
    false_neg = np.array([len(signal[signal > cut]) for cut in cut])
    return true_pos / (true_pos + false_neg)

In [ ]:
signal = projection_0
noise = projection_1

lam_cut = np.linspace(-6, 5, 100)
plt.plot(lam_cut, precision(signal, noise, lam_cut), 
         label = 'Reinheit')
plt.plot(lam_cut, recall(signal, noise, lam_cut), 
         label = 'Effizienz')
plt.legend()

__f)__ Untersuche Signal-Untergrundverhältnis:

In [ ]:
def signal_noise_ratio(signal, noise, cut):
    return np.array([len(signal[signal <= cut]) / len(noise[noise <= cut]) for cut in cut])

In [ ]:
lam_cut = np.linspace(min(noise), 2, 1000)
plt.plot(lam_cut, signal_noise_ratio(signal, noise, lam_cut), 
         label = 'Signal-Untergrundverhältnis')

lam_cut[np.argmax(signal_noise_ratio(signal, noise, lam_cut))]

__g)__ Untersuche Signifikanz:

In [ ]:
def sig(signal, noise, cut):
    return np.array([len(signal[signal <= cut]) / 
                     np.sqrt(len(noise[noise <= cut]) + len(signal[signal <= cut])) for cut in cut])

In [ ]:
lam_cut = np.linspace(-5, 5, 1000)
plt.plot(lam_cut, sig(signal, noise, lam_cut), 
         label = 'Signifikanz')

lam_cut[np.argmax(sig(signal, noise, lam_cut))]